In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from code_llama_rag.context_builder import ContextBuilder
from code_llama_rag.code_talker import CodeTalker

In [3]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [4]:
# instantiate code-llama in 4 bit mode
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.bfloat16,
)
llm_model_id = "codellama/CodeLlama-7b-Instruct-hf"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id, 
    device_map="auto",       
    quantization_config=bnb_config,         
).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
type(llm_tokenizer)

transformers.models.code_llama.tokenization_code_llama_fast.CodeLlamaTokenizerFast

In [6]:
# instantiate the context builder
folder = "example-code/"
context_builder = ContextBuilder(folder=folder)

In [7]:
# instantiate the code talker
talker = CodeTalker(llm_model, llm_tokenizer, context_builder)

In [8]:
# define the question
question = "'spell_?book.py': What is the purpose of the function abracadabra?"
# get the answer
answer = talker.answer(question)
print(answer)

number of characters in prompt: 593
number of tokens in prompt: 210
chars per token: 2.823809523809524
The purpose of the function abracadabra is to reverse the order of spells in the spellbook.



In [9]:
question = "What is the purpose of the function abracadabra?"
answer = talker.answer(question)
print(answer)

number of characters in prompt: 575
number of tokens in prompt: 201
chars per token: 2.8606965174129355
The purpose of the function abracadabra is to reverse the order of spells in the spellbook.



In [10]:
question = "List all classes with their functions in a markdown table"
answer = talker.answer(question)

number of characters in prompt: 1050
number of tokens in prompt: 346
chars per token: 3.0346820809248554


In [11]:
printmd(answer)

| Class | Function |
| --- | --- |
| Spellbook | abracadabra |
| Spellbook | list_spells |
| Kitchen | sousChef |
| Kitchen | list_ingredients |


In [12]:
question = "'^class':List all classes with their functions in a markdown table"
answer = talker.answer(question)
printmd(answer)

number of characters in prompt: 1632
number of tokens in prompt: 520
chars per token: 3.1384615384615384


| Class | Functions |
| --- | --- |
| Spellbook | abracadabra, list_spells |
| Gadget | overclock, check_status |
| Kitchen | sousChef, list_ingredients |


In [13]:
question = "List all file names"
answer = talker.answer(question)
print(answer)

number of characters in prompt: 158
number of tokens in prompt: 62
chars per token: 2.5483870967741935




In [14]:
question = "'.*.py':List all file names"
answer = talker.answer(question)
print(answer)

number of characters in prompt: 1593
number of tokens in prompt: 513
chars per token: 3.1052631578947367
['example-code/magic/spell_book.py', 'example-code/tech/gadget.py', 'example-code/food/kitchen.py']



In [15]:
question = "'.*.py\\b':List all file names and briefly describe the contents"
answer = talker.answer(question)
printmd(answer)

number of characters in prompt: 1629
number of tokens in prompt: 520
chars per token: 3.1326923076923077


The file names are:

* `magic/spell_book.py`
* `tech/gadget.py`
* `food/kitchen.py`

The contents of each file are:

* `magic/spell_book.py`: A class `Spellbook` that has a list of spells and two methods: `abracadabra` and `list_spells`. `abracadabra` reverses the order of spells in the spellbook, and `list_spells` returns a string of all spells in the spellbook.
* `tech/gadget.py`: A class `Gadget` that has a model and a performance level. `overclock` increases the performance level of the gadget, and `check_status` returns a string with the model and performance level of the gadget.
* `food/kitchen.py`: A class `Kitchen` that has a list of ingredients. `sousChef` chooses all ingredients and prepares them finely, and `list_ingredients` returns a string of all ingredients ready for cooking.
